## Load Langchain

In [3]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts.prompt import PromptTemplate
from langchain.callbacks import get_openai_callback
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory


import os
import pickle

## Convert PDF files in Domino Dataset to embeddings for semantic search

In [4]:
loader = DirectoryLoader("/mnt/data/healthcareplandetails")
data = loader.load()

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [6]:
# Chunk your data up into smaller documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)
texts[1:3]

[Document(page_content='Northwind Health Plus is a comprehensive plan that provides comprehensive coverage for medical, vision, and dental services. This plan also offers prescription drug coverage, mental health and abuse coverage, and coverage for preventive care services. With Northwind Health Plus, you can choose from a variety of in-network providers, including primary care physicians, specialists, hospitals, and pharmacies. This plan also offers coverage for emergency services, both in-network and out-of-network.\n\nNorthwind Standard', metadata={'source': '/mnt/data/healthcareplandetails/Benefit_Options.pdf'}),
 Document(page_content='Northwind Standard is a basic plan that provides coverage for medical, vision, and dental services. This plan also offers coverage for preventive care services, as well as prescription drug coverage. With Northwind Standard, you can choose from a variety of in-network providers, including primary care physicians, specialists, hospitals, and pharmac

In [10]:
print (f'There are now {len(texts)} chunks of documents')

There are now 577 chunks of documents


In [12]:
#Create embeddings of your documents to get ready for semantic search

# Read your OpenAI key from the environment
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') 
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [ ]:
# Index and store the embeddings locally in a pickle file
store = FAISS.from_texts([t.page_content for t in texts], embeddings)
with open("healthcareplandetails.pkl", "wb") as f:
    pickle.dump(store, f)

In [3]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [4]:
template = """You are an AI assistant for answering questions about information in health care plan documentation.
You are given the following extracted parts of a long document and a question. Provide a conversational answer.
If you don't know the answer, just say "Hmm, I'm not sure." Don't try to make up an answer.
If the question is not about health care policy coverage, politely inform them that you are tuned to only answer questions about the finance industry.
Question: {question}
=========
{context}
=========
Answer in Markdown:"""
QA_PROMPT = PromptTemplate(template=template, input_variables=["question", "context"])

In [5]:
# Load the embeddings from the pickle file; change the location if needed
if 'store' not in locals() or store is None:
    with open("healthcareplandetails.pkl", "rb") as f:
        store = pickle.load(f)
        

In [6]:
def get_chat_history(inputs) -> str:
    res = []
    for human, ai in inputs:
        res.append(f"Human:{human}\nAI:{ai}")
    return "\n".join(res)

In [8]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), store.as_retriever(), memory=memory, qa_prompt=QA_PROMPT,
                                                     condense_question_prompt=CONDENSE_QUESTION_PROMPT, get_chat_history=get_chat_history)

In [9]:
with get_openai_callback() as cb:
    while True:
        print("Human:")
        question = input()
        if question.lower() == "quit()":
            question = None
            break
        if question.lower() == "clear_history()":
            qa.memory.clear()
            question = None
            continue
        if question is not None and question != "" :
            print("AI:")
            print(qa.run(question))
                
print(f"Total Tokens: {cb.total_tokens}")

Human:


 What does my vision coverage provide?


AI:


Northwind Health Plus provides coverage for vision services, including preventive care services such as routine physicals, well-child visits, immunizations, and other preventive care services. It also covers emergency services, both in-network and out-of-network. Co-pays, deductibles, and out-of-pocket maximums may apply to your plan.
Human:


 What is my co-pay for contact lenses? 


AI:

The copay for contact lenses under Northwind Health Plus will depend on the type of service you receive, whether the service is rendered by an in-network or out-of-network provider, and other factors. You can find more information about cost-sharing arrangements on the Northwind Health website.
Human:


 Does my plan cover annual eye exams?


AI:

The Northwind Standard plan covers one routine eye exam per year with no deductible.
Human:


 what about hearing?


AI:

Yes, Northwind Health Plus provides coverage for hearing services. This includes hearing tests, hearing aids and hearing aid accessories, as well as cochlear implants and bone-anchored hearing aids. The plan covers up to two hearing aids or implants per ear every three years, and up to $1,500 every year for all hearing aid services, including the hearing aid itself. However, the plan does not cover hearing care services provided by out-of-network providers.
Human:


KeyboardInterrupt: Interrupted by user